# cut cross entropy

author: [xiaodongguaAIGC](!https://github.com/dhcode-cpp)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Cross-Entropy

In [2]:
X = torch.tensor([1.0, 3.0, -1.2, 1.1, -0.5, -0.8]) # logits
Y = torch.tensor([0.4, 0.2, 0.1,  0.1, 0.05, 0.15])

### Cross Entropy Pytorch

$CE(X,Y) = \sum_i -\log(\text{softmax}(X))$

In [3]:
loss = nn.CrossEntropyLoss()
l = loss(X,Y)
print(l)

tensor(2.4569)

### Cross Entropy with log softmax

when we calculate cross entropy, we need logprob

In [4]:
l = -Y * F.log_softmax(X, dim = 0 )
l = l.sum()
print(l)

tensor(2.4569)

In [5]:
p = F.softmax(X, dim = 0 )
l = -Y * p.log()
l = l.sum()
print(l)

tensor(2.4569)

## Softmax

#### forward

In [6]:
p = torch.exp(X) / (torch.exp(X)).sum()
print(p)
l = - Y * p.log()
l = l.sum()
print(l)

tensor([0.1001, 0.7394, 0.0111, 0.1106, 0.0223, 0.0165])

tensor(2.4569)

### gradient

In [7]:
N = 6 
dX = (p - Y) / 6
print(dX)

tensor([-0.0500,  0.0899, -0.0148,  0.0018, -0.0046, -0.0222])

In [8]:
dpdX = torch.diag(p) - torch.outer(p, p)
dX = -(Y / p) @ dpdX   # first term is (- Y log p)' = (-Y/p)
print(dX)

tensor([-0.2999,  0.5394, -0.0889,  0.0106, -0.0277, -0.1335])

## Log Softmax

### forward

In [9]:
LSE = torch.log( torch.sum( torch.exp( X ) ) )
p = X - LSE # X - LSE
print(p)
l = - Y * p # delete log
l = l.sum()
print(l)

tensor([-2.3019, -0.3019, -4.5019, -2.2019, -3.8019, -4.1019])

tensor(2.4569)

### gradient

首先，L = -∑y_i * log(softmax(x_i))

= -∑y_i * p_i (因为p_i = LogSoftmax(x_i))

求导链式法则：

∂L/∂x_k = ∑_i (∂L/∂p_i * ∂p_i/∂x_k)

其中：

∂L/∂p_i = -y_i

∂p_i/∂x_k = δ_{ik} - softmax(x_k) (上一问的结果)

代入得到：

∂L/∂x_k = ∑_i (-y_i * (δ_{ik} - softmax(x_k)))

= -y_k + ∑_i y_i * softmax(x_k)

= -y_k + softmax(x_k) * ∑_i y_i

= softmax(x_k) - y_k (因为∑y_i = 1)


In [10]:
l = - Y * p / N
# p.exp() is softmax result probability
dX = p.exp() - Y # lse is easy revearse p 
print(dX)

tensor([-0.2999,  0.5394, -0.0889,  0.0106, -0.0277, -0.1335])

## Cut Cross Entropy

think about llm model has a big vocabulary, so the classifer project d-dimension to |V|-dimension, and |V| usually is big like 128,000 or 256,000.

classifier proj is big GEMM when the model params < 2B

In [11]:
v = 128
d = 16
bs = 8
block_size = 4
n_b = bs // block_size
d_b = d // block_size
v_b = v // block_size

print(n_b)
print(d_b)
print(v_b)

2

4

32

In [12]:
C = torch.randn(d, v)
E = torch.randn(bs, d)
Y = torch.randn(bs, v)
Y = torch.softmax(Y, dim = 1)
print(Y.shape) # label

torch.Size([8, 128])

### standard CE

In [13]:
# ce
logits = E @ C 
print(logits.shape)
print(Y.shape)
l = loss(logits, Y)
print(l)

torch.Size([8, 128])

torch.Size([8, 128])

tensor(11.3531)

In [14]:
# softmax -> ce
P = F.softmax(logits, dim = 1)
print(P)
l = -Y * P.log()
print(l.sum() / bs)

tensor([[6.0015e-06, 2.2714e-06, 1.8833e-08,  ..., 4.1099e-08, 2.5148e-04,
         5.8345e-06],
        [3.4295e-04, 2.6087e-06, 2.6068e-04,  ..., 2.9111e-03, 9.9335e-04,
         1.7892e-03],
        [3.6669e-06, 2.7905e-05, 1.6177e-05,  ..., 1.1876e-05, 1.1016e-06,
         5.5356e-05],
        ...,
        [3.3739e-04, 5.2863e-05, 1.2740e-04,  ..., 1.5014e-05, 1.9382e-07,
         7.0755e-06],
        [1.5546e-09, 1.0746e-05, 5.3797e-05,  ..., 1.5173e-07, 3.1235e-07,
         2.4252e-05],
        [2.4066e-06, 1.0805e-03, 6.7167e-05,  ..., 6.4321e-05, 2.1308e-05,
         5.5685e-05]])

tensor(11.3531)

In [15]:
# logsoftmax -> ce
P = F.log_softmax(logits, dim = 1)
l = -Y * P
print(l.sum() / bs)

tensor(11.3531)

### Cut Cross Entropy 

#### forward Gemm

our target is get  


logprob = X @ C - log \sum_j X[i,:] C[:,j] # row sub

the X[i,1] C[1,j] + X[i,2] C[2,j] + X[i,d] C[d,j] + 

output = -Y * logprob

In [16]:
X = E @ C

In [17]:
print(X.shape)

torch.Size([8, 128])

In [18]:
X_blocks = torch.split(X, 64, dim = 1)
print(X_blocks[0].shape)
print(len(X_blocks))

torch.Size([8, 64])

2

In [19]:
E_blocks = torch.split(E, n_b, dim = 0)
C_blocks = torch.split(C, v_b, dim = 1)

print(len(E_blocks), len(C_blocks))
print(E_blocks[0].shape)
print(C_blocks[0].shape)

4 4

torch.Size([2, 16])

torch.Size([16, 32])

In [20]:
stand_mm = E @ C
print(stand_mm[:5,:5])

tensor([[ 3.0742,  2.1026, -2.6899,  2.9976, -7.5066],
        [ 1.0479, -3.8309,  0.7736, -3.0911,  0.1685],
        [-1.5154,  0.5141, -0.0311, -3.2252,  0.2030],
        [ 0.5123,  0.7709,  1.0622,  1.2311,  2.5214],
        [ 4.7593, -1.5436, -1.4017,  0.5432,  5.5318]])

In [21]:
# CCE : Algorithm 1

O = torch.zeros(bs, v)
O_blocks = torch.split(O, n_b, dim = 0)
O_result = []

for e, o in zip(E_blocks, O_blocks): # i
    e_d_blocks = torch.split(e, d_b, dim = 1)
    c_blocks = torch.split(C, d_b, dim = 0)
    for ed, cd in zip(e_d_blocks, c_blocks): # 
        p = ed @ cd
        o = o + p
    O_result.append(o)

O_mat = torch.cat(O_result, dim = 0)
print(O_mat.shape)

print(O_mat[:5,:5])

torch.Size([8, 128])

tensor([[ 3.0742,  2.1026, -2.6899,  2.9976, -7.5066],
        [ 1.0479, -3.8309,  0.7736, -3.0911,  0.1685],
        [-1.5154,  0.5141, -0.0311, -3.2252,  0.2030],
        [ 0.5123,  0.7709,  1.0622,  1.2311,  2.5214],
        [ 4.7593, -1.5436, -1.4017,  0.5432,  5.5318]])

#### forward logsoftmax

Original LSE

In [22]:
A = E @ C 
LSE = torch.log( torch.sum( torch.exp(A), dim = 1, keepdim=True ) )
print(LSE)

tensor([[15.0977],
        [ 9.0258],
        [11.0008],
        [ 9.5224],
        [12.4312],
        [ 9.7486],
        [14.1608],
        [ 9.5511]])

In [23]:
# CCE : Algorithm 2

LSE = torch.ones(bs, 1) * -100000.0 # -inf

# LSE part
for n, e in enumerate(E_blocks):
    for v, c in enumerate(C_blocks):
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd
        LSE_nv = torch.log( torch.sum( torch.exp(A), dim = 1, keepdim=True ) )
        LSE[n_b * n : n_b * (n+1)] = torch.log( torch.exp(LSE[n_b * n : n_b * (n+1)]) 
                                            + torch.exp(LSE_nv)
                                            )
print(LSE.shape)
print(LSE)

torch.Size([8, 1])

tensor([[15.0977],
        [ 9.0258],
        [11.0008],
        [ 9.5224],
        [12.4312],
        [ 9.7486],
        [14.1608],
        [ 9.5511]])

#### Backward

In [24]:
v = 128
d = 16
bs = 8
block_size = 4
n_b = bs // block_size
d_b = d // block_size
v_b = v // block_size

print(n_b)
print(d_b)
print(v_b)

2

4

32

#### standard backward

In [25]:
# basic backward
# P = softmax(E @ C, dim = 1)
# loss = - Y * torch.log(P)

Y = torch.randn(bs, v)
Y = torch.softmax(Y, dim = 1)

X = E @ C
P = torch.softmax(X, dim = 1)

loss = (- Y * torch.log(P)).mean()
print(loss)

tensor(0.0914)

In [26]:
# Stand backward
N = bs * v

dP = (P - Y) / N
dPdX = torch.ones(v, v)
for i in range(bs):
    dPdX += torch.diag(P[i,:]) - torch.outer(P[i,:], P[i,:]) 

dX = dP @ dPdX 
print(dX.shape)

dE = dP @ C.t()
dC = E.t() @ dP 

print(dE[:5,:5])
print(dC[:5,:5])


torch.Size([8, 128])

tensor([[-3.0774e-06,  1.6830e-03, -1.3377e-03, -1.8160e-04, -3.0330e-04],
        [-9.2404e-04, -8.7230e-04, -1.0772e-03, -1.4970e-05, -5.5368e-04],
        [-3.1002e-04,  4.5423e-04,  1.7460e-04, -5.0915e-04, -6.3905e-04],
        [-7.9295e-04, -2.0689e-03, -6.9289e-04, -7.2696e-04,  1.1507e-03],
        [-1.9385e-03, -2.7085e-03, -7.6056e-04, -4.4964e-04, -4.5431e-04]])

tensor([[ 3.2876e-05,  1.1341e-04,  2.4314e-05,  2.4401e-05,  9.0846e-05],
        [-1.6622e-05,  2.6949e-05,  2.7279e-05,  2.2369e-05,  3.7877e-06],
        [ 1.0191e-05,  4.7310e-05,  6.9686e-06,  1.1149e-06,  2.8567e-05],
        [ 2.4554e-05,  8.1344e-05,  2.4788e-05,  1.9775e-05,  3.3926e-05],
        [ 7.2091e-06,  2.9094e-05,  8.7493e-06,  2.1335e-05, -1.6757e-05]])

#### LSE Backward

In [27]:
X = E @ C
LSE = torch.log(torch.sum( torch.exp(X), dim = 1, keepdim = True))
print(LSE)
P = X - LSE # 逐行减去LSE
loss = (- Y * P) # (-Y * X + Y * LSE)
print(loss.mean() )


tensor([[15.0977],
        [ 9.0258],
        [11.0008],
        [ 9.5224],
        [12.4312],
        [ 9.7486],
        [14.1608],
        [ 9.5511]])

tensor(0.0914)

In [28]:
dY_dX = -Y / N # P = "X" - LSE
dY_dLSE = torch.sum(Y / N, dim = 1, keepdim = True) # P = X - "LSE"
dLSE_dX = F.softmax(X, dim = 1)
dY_dX_LSE = dY_dLSE * dLSE_dX

dY_dX_total =  dY_dX + dY_dX_LSE

dE = dY_dX_total @ C.t()
dC = E.t() @ dY_dX_total 

print(dE[:5,:5])
print(dC[:5,:5])

tensor([[-3.0774e-06,  1.6830e-03, -1.3377e-03, -1.8160e-04, -3.0330e-04],
        [-9.2404e-04, -8.7230e-04, -1.0772e-03, -1.4970e-05, -5.5368e-04],
        [-3.1002e-04,  4.5423e-04,  1.7460e-04, -5.0915e-04, -6.3905e-04],
        [-7.9295e-04, -2.0689e-03, -6.9289e-04, -7.2696e-04,  1.1507e-03],
        [-1.9385e-03, -2.7085e-03, -7.6056e-04, -4.4964e-04, -4.5431e-04]])

tensor([[ 3.2876e-05,  1.1341e-04,  2.4314e-05,  2.4401e-05,  9.0846e-05],
        [-1.6622e-05,  2.6949e-05,  2.7279e-05,  2.2369e-05,  3.7877e-06],
        [ 1.0191e-05,  4.7310e-05,  6.9686e-06,  1.1149e-06,  2.8567e-05],
        [ 2.4554e-05,  8.1344e-05,  2.4788e-05,  1.9775e-05,  3.3926e-05],
        [ 7.2091e-06,  2.9094e-05,  8.7493e-06,  2.1335e-05, -1.6757e-05]])

In [29]:
dE_LSE = dLSE_dX @ C.t()
dC_LSE = E.t() @ dLSE_dX 
print(dE_LSE[:5,:5])
print(dC_LSE[:5,:5])

tensor([[ 0.2161,  1.9069, -1.5344, -0.2329, -0.2667],
        [-0.7757, -0.5801, -1.1768, -0.0819, -0.5917],
        [-0.2083,  0.5824,  0.0260, -0.7244, -0.5414],
        [-0.4269, -1.8212, -0.8554, -0.5496,  1.0672],
        [-1.7580, -2.5212, -0.7580, -0.5546, -0.3271]])

tensor([[-1.4649e-03, -1.1066e-03, -7.3092e-04, -8.6530e-05, -1.1256e-03],
        [-1.2147e-03,  2.8367e-04, -2.3699e-04, -3.7976e-04, -3.6783e-03],
        [-4.3246e-04,  2.8613e-04, -2.0724e-04, -2.2863e-06, -1.5024e-04],
        [-9.7449e-04, -5.1255e-04, -3.5255e-04, -9.0179e-05, -1.4561e-03],
        [ 1.4869e-04, -1.3156e-03, -1.9962e-04, -4.2939e-05,  5.8839e-05]])

#### Pytorch AutoGrad LSE backward

 仅检验手动计算的梯度是否正确

In [30]:
# # 注意要清除梯度，得到的才准确
# # E.grad.zero_()
# # C.grad.zero_()
# # Y.grad.zero_()

# E.requires_grad = True
# C.requires_grad = True
# Y.requires_grad = True


# X = E @ C
# X.retain_grad()

# LSE = torch.log(torch.sum( torch.exp(X), dim = 1, keepdim = True))
# LSE.retain_grad()

# P = X - LSE # 逐行减去LSE
# P.retain_grad()

# loss = (- Y * P) # (-Y * X + Y * LSE)
# loss.retain_grad()
# # print(loss)

# l = loss.mean()
# l.backward() 

In [31]:
# print(loss.grad[:5,:5])

# print(LSE.grad[:5,:5])
# print(LSE.grad.shape)

# print(X.grad[:5,:5])
# print(E.grad[:5,:5])
# print(C.grad[:5,:5])

#### CCE LSE Backward

In [32]:
# CCE : Algorithm 3

is_filter = False
epsilon = 1e-6

dE = torch.zeros_like(E)
dC = torch.zeros_like(C)

dE_blocks = torch.split(dE, n_b, dim = 0)
dC_blocks = torch.split(dC, v_b, dim = 1)

print(Y.shape)
d_LSE = dY_dLSE
print(d_LSE)

# LSE part
for n, e in enumerate(E_blocks):
    for v, c in enumerate(C_blocks):
        
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd # A_nv

        S = torch.exp(A - LSE[n_b * n : n_b * (n+1)]) # softmax

        # filter S
        if is_filter:
            if  (S < epsilon).all():
                print('skip', n, v)
                continue
            
        
        d_LSE_block = d_LSE[n_b * n : n_b * (n+1), :]
        d_index = 0
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            dE_blocks[n][ :, d_index * d_b: (d_index+1)*d_b ] +=  (d_LSE_block * S) @ cd.t()
            dC_blocks[v][ d_index * d_b: (d_index+1)*d_b, : ] +=  ed.t() @ (d_LSE_block * S)
            d_index = d_index + 1


# dLSE gradient part
dE_part_LSE = torch.cat(dE_blocks, dim = 0)
dC_part_LSE = torch.cat(dC_blocks, dim = 1)

print(dE_part_LSE.shape)
print(dC_part_LSE.shape)

print(dE_part_LSE[:5,:5])
print(dC_part_LSE[:5,:5])

torch.Size([8, 128])

tensor([[0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010],
        [0.0010]])

torch.Size([8, 16])

torch.Size([16, 128])

tensor([[ 2.1101e-04,  1.8622e-03, -1.4984e-03, -2.2748e-04, -2.6046e-04],
        [-7.5749e-04, -5.6646e-04, -1.1492e-03, -8.0008e-05, -5.7780e-04],
        [-2.0341e-04,  5.6872e-04,  2.5342e-05, -7.0742e-04, -5.2867e-04],
        [-4.1686e-04, -1.7785e-03, -8.3539e-04, -5.3669e-04,  1.0422e-03],
        [-1.7168e-03, -2.4621e-03, -7.4023e-04, -5.4160e-04, -3.1944e-04]])

tensor([[-1.4306e-06, -1.0806e-06, -7.1379e-07, -8.4502e-08, -1.0992e-06],
        [-1.1862e-06,  2.7702e-07, -2.3143e-07, -3.7086e-07, -3.5921e-06],
        [-4.2233e-07,  2.7943e-07, -2.0238e-07, -2.2327e-09, -1.4672e-07],
        [-9.5165e-07, -5.0053e-07, -3.4428e-07, -8.8065e-08, -1.4220e-06],
        [ 1.4520e-07, -1.2848e-06, -1.9494e-07, -4.1933e-08,  5.7460e-08]])

In [33]:
# dX part dE, dC
dE_part_X = dY_dX @ C.t()
dC_part_X = E.t() @ dY_dX

# dX Total
dE_total = dE_part_X + dE_part_LSE
dC_total = dC_part_X + dC_part_LSE

print(dE_total[:5, :5])
print(dC_total[:5, :5])

tensor([[-3.0775e-06,  1.6830e-03, -1.3377e-03, -1.8160e-04, -3.0330e-04],
        [-9.2404e-04, -8.7230e-04, -1.0772e-03, -1.4970e-05, -5.5368e-04],
        [-3.1002e-04,  4.5422e-04,  1.7460e-04, -5.0915e-04, -6.3905e-04],
        [-7.9295e-04, -2.0689e-03, -6.9289e-04, -7.2696e-04,  1.1507e-03],
        [-1.9385e-03, -2.7085e-03, -7.6056e-04, -4.4964e-04, -4.5431e-04]])

tensor([[ 3.2876e-05,  1.1341e-04,  2.4314e-05,  2.4401e-05,  9.0846e-05],
        [-1.6622e-05,  2.6949e-05,  2.7279e-05,  2.2369e-05,  3.7877e-06],
        [ 1.0191e-05,  4.7310e-05,  6.9686e-06,  1.1149e-06,  2.8567e-05],
        [ 2.4554e-05,  8.1344e-05,  2.4788e-05,  1.9775e-05,  3.3926e-05],
        [ 7.2091e-06,  2.9094e-05,  8.7493e-06,  2.1335e-05, -1.6757e-05]])

#### CCE Backward total

In [34]:
print(Y)
Y_t = Y / 0.001 # be sharp
YY = F.softmax(Y_t, dim = 1)
print(YY)
_, label = torch.max(YY, dim = 1)
print(label)

print(YY[0, label[0]])

tensor([[0.0077, 0.0046, 0.0018,  ..., 0.0070, 0.0058, 0.0023],
        [0.0016, 0.0617, 0.0058,  ..., 0.0029, 0.0063, 0.0035],
        [0.0086, 0.0008, 0.0023,  ..., 0.0012, 0.0135, 0.0068],
        ...,
        [0.0091, 0.0054, 0.0052,  ..., 0.0153, 0.0245, 0.0024],
        [0.0036, 0.0119, 0.0071,  ..., 0.0269, 0.0027, 0.0047],
        [0.0076, 0.0016, 0.0021,  ..., 0.0147, 0.0019, 0.0011]])

tensor([[3.7805e-25, 1.6146e-26, 1.0305e-27,  ..., 1.8982e-25, 5.7151e-26,
         1.6623e-27],
        [2.3918e-27, 3.1155e-01, 1.6124e-25,  ..., 8.4964e-27, 2.4980e-25,
         1.5034e-26],
        [7.9437e-19, 3.2957e-22, 1.4669e-21,  ..., 4.8355e-22, 1.0210e-16,
         1.2394e-19],
        ...,
        [3.1144e-18, 7.5563e-20, 6.2492e-20,  ..., 1.5313e-15, 1.4256e-11,
         3.9089e-21],
        [6.5182e-25, 2.6652e-21, 2.1679e-23,  ..., 8.5875e-15, 2.5412e-25,
         1.8990e-24],
        [9.3496e-28, 2.3253e-30, 3.8035e-30,  ..., 1.2009e-24, 3.0555e-30,
         1.4015e-30]])

tensor([ 21,  89, 112, 110, 126,  74,  38,  76])

tensor(1.)

##### max likelihood version CE

In [35]:
print(P.shape)

torch.Size([8, 128])

In [36]:
print(label.unsqueeze(0))
P_CEL = torch.gather(P, dim = 1, index = label.unsqueeze(1))
loss = -P_CEL
l = loss.mean()
print(loss)
print(l)

tensor([[ 21,  89, 112, 110, 126,  74,  38,  76]])

tensor([[11.2310],
        [12.6195],
        [10.8524],
        [14.9215],
        [12.0455],
        [ 9.4452],
        [27.2964],
        [13.8197]])

tensor(14.0289)

In [37]:
loss = nn.CrossEntropyLoss()
l_torch = loss(X, label)
print(l_torch)

tensor(14.0289)

In [38]:
# CCE : Algorithm 4

bs = 8
v = 128


M = torch.zeros(bs, v)
for i in range(bs):
    M[i, label[i]] = 1
M_blocks = torch.split(M, n_b, dim = 0)

dE = torch.zeros_like(E)
dC = torch.zeros_like(C)

dE_blocks = torch.split(dE, n_b, dim = 0)
dC_blocks = torch.split(dC, v_b, dim = 1)

d_LCE = torch.ones(bs, 1) / bs

# LSE part
for n, e in enumerate(E_blocks):
    
    M_v_blcoks = torch.split(M_blocks[n], v_b, dim = 1)
                                 
    for v, c in enumerate(C_blocks):
        
        A = torch.zeros(n_b, v_b)
        e_d_blocks = torch.split(e, d_b, dim = 1)
        c_d_blocks = torch.split(c, d_b, dim = 0)
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            A += ed @ cd # A_nv


        S = torch.exp(A - LSE[n_b * n : n_b * (n+1)]) # softmax
        G = - M_v_blcoks[v] + S
        
        
        # filter S
        if is_filter:
            if  (S < epsilon).all():
                print('skip',n,v)
                continue
        
        d_LCE_block = d_LCE[n_b * n : n_b * (n+1), :]
        d_index = 0
        for ed, cd in zip(e_d_blocks, c_d_blocks):
            dE_blocks[n][ :, d_index * d_b: (d_index+1)*d_b ] +=  (d_LCE_block * G) @ cd.t()
            dC_blocks[v][ d_index * d_b: (d_index+1)*d_b, : ] +=  ed.t() @ (d_LCE_block * G)
            d_index = d_index + 1


# dLSE gradient part
dE = torch.cat(dE_blocks, dim = 0)
dC = torch.cat(dC_blocks, dim = 1)

print(dE[:5,:5])
print(dC[:5,:5])

tensor([[-0.0667,  0.5227, -0.0998,  0.1277, -0.0061],
        [-0.0786,  0.0198, -0.1159,  0.0683, -0.2388],
        [ 0.0438,  0.0172, -0.0514, -0.1818, -0.0870],
        [-0.1582, -0.4584, -0.1845, -0.1854,  0.2185],
        [-0.4194, -0.2052, -0.1018, -0.0136, -0.1755]])

tensor([[-1.8311e-04, -1.3832e-04, -9.1366e-05, -1.0816e-05, -1.4070e-04],
        [-1.5184e-04,  3.5458e-05, -2.9624e-05, -4.7469e-05, -4.5979e-04],
        [-5.4058e-05,  3.5767e-05, -2.5905e-05, -2.8578e-07, -1.8780e-05],
        [-1.2181e-04, -6.4068e-05, -4.4068e-05, -1.1272e-05, -1.8201e-04],
        [ 1.8586e-05, -1.6446e-04, -2.4953e-05, -5.3674e-06,  7.3549e-06]])